# American Express - Default Prediction

## Description du sujet
American Express a pour objectif de prédire la probabilité qu'un client ne rembourse pas le solde de sa carte de crédit.  
Pour cela, nous avons accès à une fenêtre de 18 mois après le dernier relevé de carte de crédit de chaque client.  
Le client est étiqueté en défaut lorsqu'au bout de 120 jours après le dernier relevé de sa carte de crédit, il n'a pas remboursé son solde.

Notre but ici, sera d'analyser les données pour mieux comprendre le sujet.

## Description des variables

D_* = Delinquency variables  
S_* = Spend variables  
P_* = Payment variables  
B_* = Balance variables  
R_* = Risk variables

In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

# Listing des données disponibles
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Nombre de données à charger pour l'analyse
NROWS = 100_000

In [ ]:
train_data = pd.read_csv("/kaggle/input/amex-default-prediction/train_data.csv", nrows=NROWS)
train_labels = pd.read_csv("/kaggle/input/amex-default-prediction/train_labels.csv", nrows=NROWS)

In [ ]:
categorical_variables = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

# Définition des variables en variables qualitatives
train_data[categorical_variables] = train_data[categorical_variables].astype('category')

# On récupère toutes les variables quantitatives
numeric_variables = train_data._get_numeric_data().columns

# On récupère toutes les variables qualitatives
categorical_variables = train_data.select_dtypes(exclude=['number', 'object']).columns

In [ ]:
train_data.describe()

In [ ]:
train_data = train_data.loc[:,(train_data.isna().sum().sort_values() < 10_000)]

# On met à jour toutes les variables quantitatives suite aux nettoyage des données
numeric_variables = train_data._get_numeric_data().columns

train_data[numeric_variables] = train_data[numeric_variables].fillna(train_data[numeric_variables].std().to_dict())

In [ ]:
# colonne à supprimer
# train_data = train_data.drop('D_87', axis=1)

In [ ]:
train_data.describe()

### Après le nettoyage

10 variables qualitatives  
139 variables quantitatives  
2 variables de type objet (customer_id et date)

In [ ]:
train_data.info()

### Un client peut avoir 1 à plusieurs enregistrements

In [ ]:
print("Moyenne des enregistrement par client : ", train_data.customer_ID.value_counts().mean())
print("Médiane des enregistrement par client : ", train_data.customer_ID.value_counts().std())

### Nombre de clients en défaut ou non
73959 clients en règle  
26041 clients en défaut de paiement

In [ ]:
train_labels.target.value_counts()

In [ ]:
# train_data['customer_ID'] = train_data['customer_ID'].astype('object')
# train_labels['customer_ID'] = train_labels['customer_ID'].astype('object')

In [ ]:
train_data_labeled = train_data.merge(train_labels, on='customer_ID')

In [ ]:
target_corr = train_data_labeled.corr()['target']
corr_col = ((target_corr.sort_values() > 0.3) + (target_corr.sort_values() < -0.3)).to_dict()
corr_col['customer_ID'] = True

### Analyse des clients

In [ ]:
clients_regle = train_data_labeled[train_data_labeled['target'] == 0][corr_col]

In [ ]:
print("Moyenne des enregistrement des clients en règle : ", clients_regle.customer_ID.value_counts().mean())
print("Médiane des enregistrement des clients en règle : ", clients_regle.customer_ID.value_counts().std())

In [ ]:
clients_defaut = train_data_labeled[train_data_labeled['target'] == 1][corr_col]

In [ ]:
print("Moyenne des enregistrement des clients en défaut : ", clients_defaut.customer_ID.value_counts().mean())
print("Médiane des enregistrement des clients en défaut : ", clients_defaut.customer_ID.value_counts().std())

Le nombre d'enregistrement par client ne semble pas influencer le fait qu'ils seront en défaut ou non.

In [ ]:
client_defaut = train_data_labeled[train_data_labeled['customer_ID'] == '0000f99513770170a1aba690daeeb8a96da4a39f11fc27da5c30a79db61c1e85']

In [ ]:
delinquency_col = [col for col in clients_defaut.columns if col.startswith('D_')]
spend_col = [col for col in clients_defaut.columns if col.startswith('S_')]
payment_col = [col for col in clients_defaut.columns if col.startswith('P_')]
balance_col = [col for col in clients_defaut.columns if col.startswith('B_')]
risk_col = [col for col in clients_defaut.columns if col.startswith('R_')]

variables_col = [delinquency_col, spend_col, payment_col, balance_col, risk_col]

In [ ]:
clients_defaut.describe()

In [ ]:
clients_regle.describe()

# Etude des variables

In [ ]:
import matplotlib.pyplot as plt


def subplots(columns):
    """
    Fonction permettant d'afficher la densité de chaque variables en fonction de la solvabilité du client
    
    Target = 0 : Client solvable
    Target = 1 : Client en défaut de paiement
    
    Parameters
    ----------
    columns : array
        Liste des variables à afficher
    """

    # calcul de la dimension de la grille
    size = len(columns)
    cols = 5
    rows = size // cols
    rows += size % cols
    
    # initialisation de la figure
    fig = plt.figure(figsize=(20,30), constrained_layout = True)
    
    position = range(1,size + 1)

    for key, col_name in enumerate(columns, start=0) :
        
        ax = fig.add_subplot(rows,cols,position[key])
        clients_regle[col_name].plot(kind='kde',ax=ax, label="0")
        clients_defaut[col_name].plot(kind='kde',ax=ax, label="1")

        ax.legend(title="Target")
        ax.set_title(col_name)

## Variables de délinquance 

On observe les différences entre les clients solvables et les clients en défaut de paiement afin de découvrir de potentielles variables discriminantes.

In [ ]:
subplots(delinquency_col)

## Variables de dépense

In [ ]:
subplots(spend_col)

## Variables de solde

In [ ]:
subplots(balance_col)

## Variables de paiement

In [ ]:
subplots(payment_col)

## Variables de risque

In [ ]:
subplots(risk_col)